![人工智慧 - 自由團隊](https://raw.githubusercontent.com/chenkenanalytic/img/master/af/aifreeteam.png)


<center>Welcome to the course《Python: from business analytics to Artificial Intelligence》by AI . FREE Team.</center>
<br>
<center>歡迎大家來到 AI . FREE Team 《Python 從商業分析到人工智慧》的 AI 基礎教學 - Lesson 07 Sentiment Analysis (3-07)。 </center>
<br>

<center>(Author: Chen Ken；Date of published: 2023/08/07；AI . FREE Team Website: https://ai-free-team.github.io/)</center>

# 情感分析 - Sentiment Analysis

在認識完電腦視覺於二元分類、多元分類的應用後，我們將踏入自然語言領域：進行文本情感分析，應用 AI 多元分類進行影評分數預測，同時理解不同字詞的出現，會如何影響影評分數。

本篇實作將帶領讀者參與 Kaggle Sentiment Analysis on Movie Reviews 競賽，並透過實作認識自然語言領域。

## STEP 0. 註冊 Kaggel 帳號 & 下載影評資料集

Kaggle Rotten Tomatoes dataset 爛番茄資料集網址： https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

※ 下載完資料集，並上傳、解壓縮 (或是透過google drive連結，每次使用此教學即可避免重複上傳)

In [ ]:
### 資料上傳時間，依電腦網速而定 (共三個壓縮檔)
from google.colab import files
uploaded = files.upload()

In [ ]:
### 解壓縮資料集，路徑請依據實際情況調整
!unzip "/content/sentiment-analysis-on-movie-reviews.zip" -d "/content"
!unzip "/content/test.tsv.zip" -d "/content"
!unzip "/content/train.tsv.zip" -d "/content"

## STEP 1. 讀取資料集

使用 pandas 套件讀取 tsv 檔。

In [ ]:
### 導入 pandas 套件
import pandas as pd

### 讀取訓練資料集與測試資料集
train_raw = pd.read_csv('/content/train.tsv', sep='\t')
test_raw = pd.read_csv('/content/test.tsv', sep='\t')

### 檢視資料集

In [ ]:
train_raw.head(5)

## STEP 2. 文字資料集清理

在進行自然語言處理任務時，辨識文句語意過程中，無意義用詞(stopword)、大寫英文字、特殊字符、標點符號，常常容易影響準確率，因此在本實作教學，我們將採用文字清理的常見作法，將對應的字句內文進行處理。

In [ ]:
# 導入相關文字清理套件

### beautifulsoup 為常見的爬蟲使用套件，擅長解析網頁原始碼 (ref: https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
from bs4 import BeautifulSoup

### re 套件為正規表達式套件 (ref: https://docs.python.org/3/library/re.html)
import re

### nltk 為自然語言處理常用套件 (ref: https://www.nltk.org/)
import nltk
from nltk.corpus import stopwords # 導入停用詞模組
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# 自定義文本清理的函數
def Preprocessing(text):

  # 將全部字詞轉為小寫
  text = text.lower()

  # 移除網頁編碼相關內容，例如：html原始碼
  text = BeautifulSoup(text).get_text()

  # 移除特殊字元
  text = re.sub("[^a-zA-Z]"," ", text)

  # 使用 nltk 套件，將文句切割成字詞列表
  wordList = nltk.word_tokenize(text)

  # # 使用字詞列表，刪除停用詞 -- 不使用
  # filtered = [w for w in wordList if w not in stopwords.words('english')]

  return " ".join(wordList)

In [ ]:
# 使用 for loop 清理文本資料 (執行約數分鐘)
for i in range(len(train_raw['Phrase'])):
  train_raw['Phrase'][i] = Preprocessing(train_raw['Phrase'][i])

In [ ]:
# 快速檢視資料集
train_raw

## STEP 3. 建立字詞索引 (Word Indexing)

在進行自然語言訓練任務時，考量到神經網路的運算仍是以數字進行計算，因此我們必須建立數字索引對應到字詞，方能使神經網路進行訓練。

以中文字建立數值索引為例，如：｛ 一： 0 , 丁： 1 , ... ... , 龜： 3978 , ... ｝

In [ ]:
# 使用 keras 內建套件，建立字詞索引 (word indexing)
from keras.preprocessing.text import Tokenizer

# 先建立空的索引
tokenizer = Tokenizer()

# 讀取文本，找出所有單一字詞建立索引
tokenizer.fit_on_texts(train_raw['Phrase'])

# 將原始文本轉換成索引形式 (儲存在 sequences)
sequences = tokenizer.texts_to_sequences(train_raw['Phrase'])

# 將索引對應的字詞找出，計算共有幾個獨立字詞
word_index = tokenizer.word_index
print('共有 ',  len(word_index), ' 獨立字詞')

In [ ]:
# 檢視字詞索引
word_index

## STEP 4. 訓練資料集前處理

在訓練讀取自然語言的文本前，我們可以發現每筆資料的文字數量都不太一樣，因此將使用 padding 的工具，讓一筆輸入到神經模型的文本都具備一樣的長度，在進行資料切分為訓練與驗證集。

In [ ]:
# 找出最長字串
se_len = []
for i in sequences:
  se_len.append(len(i))
max(se_len)

In [ ]:
# 使用 word padding 填補每一個文本
from keras.utils import pad_sequences
data = pad_sequences(sequences, maxlen=48)

In [ ]:
# 使用 sklearn 切割資料集套件
from sklearn.model_selection import train_test_split

# 使用 one hot encoding 處理資料標籤 (評分)
labels = pd.get_dummies(train_raw['Sentiment'])

# 切割資料集
x_train, x_test, y_train, y_test = train_test_split(data, labels.to_numpy(), test_size=0.2, random_state=7, stratify=labels.to_numpy())

## STEP 5. 建置 NLP 類神經模型

In [ ]:
# 引入 tensorflow 套件
import tensorflow as tf

model = tf.keras.Sequential()

# 使用 word embedding 技術
model.add(tf.keras.layers.Embedding(15128, 128, input_length=48))

# 使用 LSTM 模型
model.add(tf.keras.layers.LSTM(64, input_shape=(None, 30),return_sequences=True))

# 使用 dropout
model.add(tf.keras.layers.Dropout(0.5))

# 使用 LSTM 模型
model.add(tf.keras.layers.LSTM(64, input_shape=(None, 30)))

# 使用 dropout
model.add(tf.keras.layers.Dropout(0.5))

# 使用 dense layer
model.add(tf.keras.layers.Dense(16, activation='relu'))

# 使用 dense layer
model.add(tf.keras.layers.Dense(5,activation="sigmoid"))

# 查看模型概況
model.summary()

## STEP 6. 模型訓練

In [ ]:
# 使用 Adam 優化器；損失函數使用分類交叉熵
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

# 開始訓練
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

## STEP 7. 測試資料清理

依據訓練資料的清理方式，進行測試資料清理。

In [ ]:
# 使用先前自訂義函數清理資料
for i in range(len(test_raw['Phrase'])):
  test_raw['Phrase'][i] = Preprocessing(test_raw['Phrase'][i])

In [ ]:
# 使用字詞索引，處理測試資料文本
test_sequences = tokenizer.texts_to_sequences(test_raw['Phrase'])

# 使用 word padding
test_data = pad_sequences(test_sequences, maxlen=48)

## STEP 8. 模型預測

使用訓練模型判讀 test 資料集，並將預測結果整理成 submit data。

In [ ]:
# 將測試資料丟入模型
prediction = model.predict(test_data)
prediction.shape

In [ ]:
# 讀取預測出來的五種評分機率，選擇機率最高的評分作為答案
answers = []

for i in range(len(prediction)):
  for j in range(len(prediction[i])):
    if prediction[i][j] == prediction[i].max():
      answers.append(j)

In [ ]:
# 讀取範例資料，並將答案填入
sample_submit = pd.read_csv("/content/sampleSubmission.csv")
sample_submit['Sentiment'] = answers
sample_submit.to_csv('answer.csv',index=False)

【optional】 下載範例成果 (csv檔案)，並上傳到 Kaggle 平台。

In [ ]:
from google.colab import files
files.download("answer.csv")

#結論與發現

透過簡單的 NLP 範例實作，可以發現特定的神經網路可以處理文本多元分類的議題，

並可以發現不同字詞、詞句的組合，AI 模型都能夠簡單地進行預測影評可能給的評分，

對於 NLP 領域來說，預測文本內容的評分(情感分析)，只是其中一塊自然語言處理的應用領域，

一般稱作 NLU (Natural Language Understanding)，而另一個領域為 NLG (Natural Language Generation)，

中文分別稱作：自然語言理解 (NLU)、自然語言生成(NLG)，

下一篇教學，將帶領讀者們認識自然語言生成專題：解夢文本生成。

# 更深入的教學與專案實作

如果你/妳對於深入開發 AI 技術的專案有興趣，想嘗試更多、更有趣、更扎實的實務AI技術專案，

歡迎參考我們從0到1的AI技術課程：[《學習 AI 一把抓：點亮人工智慧技能樹》](https://hahow.in/cr/slashie-ai-free-team)！

# 課程文件

### AI Foundations 課程清單
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_00_Preface_%E5%89%8D%E8%A8%80.ipynb">Lesson 00 Preface 課程前言</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_01_MP_model.ipynb">Lesson 01 MP model MP 模型</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_02_Logistic_Regression.ipynb">Lesson 02 Logistic Regression 羅吉斯迴歸</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_03_Neural_Network.ipynb">Lesson 03 Neural Network 類神經網路</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_04_Deep_Neural_Network.ipynb">Lesson 04 Deep Neural Network 深度神經網路</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_05_Convolution_Neural_Network.ipynb">Lesson 05 Convolution Neural Network 卷積神經網路</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_06_Handwriting_Traditional_Chinese_Characters_Recognition.ipynb">Lesson 06 Handwriting Traditional Chinese Characters Recognition 繁體中文手寫辨識</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_07_Sentiment_Analysis.ipynb">Lesson 07 Sentiment Analysis 情感分析</a> (We are here now! --本篇課程--)
- <a href="https://colab.research.google.com/github/AI-FREE-Team/AI-Introduction/blob/main/documents/AI_FREE_Team_AI_Fundations__Lesson_08_AI_Dream_Interpretation.ipynb">Lesson 08 AI Dream Interpretation AI解夢</a>